In [1]:
import pandas as pd  
import json 
import csv 
import itertools



In [2]:
Person='You'

In [3]:

with open('demo_data/js/tweet.js') as js_file:
    tweets_str = js_file.read()
    tweets_str=tweets_str[len('window.YTD.tweet.part0 = '):]
    tweets = json.loads(tweets_str)


In [4]:

with open('demo_data/js/following.js') as js_file:
    following_str = js_file.read()
    following_str=following_str[len('window.YTD.following.part0 ='):]
    followings = json.loads(following_str)



In [5]:

with open('demo_data/js/like.js') as js_file:
    like_str = js_file.read()
    like_str=like_str[len('window.YTD.like.part0 = '):]
    likes = json.loads(like_str)



In [6]:
accounts=[]

for account in followings:
    accounts.append(account['following']['accountId'])
    
following_df=pd.DataFrame()
following_df['account_id']=accounts
following_df['account']=Person

following_df.to_csv('demo_data/csv/following.csv',index=False)

In [7]:
liked_tweets=[]

for like in likes:
    liked_tweets.append(like['like']['fullText'])
    
liked_tweets_df=pd.DataFrame()
liked_tweets_df['liked_tweet']=liked_tweets

liked_tweets_df.to_csv('demo_data/csv/likes.csv',index=False)

In [8]:
tweet_text=[]
tweet_type=[]
post_time=[]
tweet_source=[]
fav_count=[]
ret_count=[]
tweet_id=[]
tweet_language=[]
hashtags=[]
retweeted_from=[]
reply_to_id=[]
reply_to_name=[]
retweeted_from_name=[]


for tweet in tweets: 
        tweet_id.append(tweet['tweet']['id'])
        tweet_language.append(tweet['tweet']['lang'])
        post_time.append(tweet['tweet']['created_at'])
        fav_count.append(tweet['tweet']['favorite_count'])
        ret_count.append(tweet['tweet']['retweet_count'])
        tweet_source.append(tweet['tweet']['source'].split('>')[1].split('<')[0])

        
        if len(tweet['tweet']['entities']['hashtags'])!=0:
            hashtags_in_tweet=[]
            for h in tweet['tweet']['entities']['hashtags']:
                hashtags_in_tweet.append(h['text'])
            hashtags.append(hashtags_in_tweet)
        else:
            hashtags.append('')
        
        if tweet['tweet']['full_text'][:1] == "@":
            tweet_text.append(tweet['tweet']['full_text'])
            tweet_type.append("reply")
            retweeted_from.append('')
            retweeted_from_name.append('')
            if len(tweet['tweet']['entities']['user_mentions'])!=0:
                reply_to_id.append(tweet['tweet']['in_reply_to_user_id_str'])
                reply_to_name.append(tweet['tweet']['entities']['user_mentions'][0]['name'])

            else:
                reply_to_id.append('')
                reply_to_name.append('')


        elif tweet['tweet']['full_text'][:3] == "RT ":
            tweet_text.append(tweet['tweet']['full_text'].split(':')[1])
            tweet_type.append("retweet")
            reply_to_id.append('')
            reply_to_name.append('')

            if len(tweet['tweet']['entities']['user_mentions'])!=0:
                retweeted_from.append(tweet['tweet']['entities']['user_mentions'][0]['id_str'])
                retweeted_from_name.append(tweet['tweet']['entities']['user_mentions'][0]['name'])
            else:
                retweeted_from.append('')
                retweeted_from_name.append('')
        else:
            tweet_text.append(tweet['tweet']['full_text'])
            tweet_type.append("tweet")
            retweeted_from.append('')
            retweeted_from_name.append('')
            reply_to_id.append('')
            reply_to_name.append('')

tweets_df=pd.DataFrame()
tweets_df['tweet_id']=tweet_id
tweets_df['tweet']=tweet_text
tweets_df['post_time']=post_time
tweets_df['type']=tweet_type
tweets_df['tweet_source']=tweet_source
tweets_df['favorite_count']=fav_count
tweets_df['retweet_count']=ret_count
tweets_df['retweeted_from_id']=retweeted_from
tweets_df['retweeted_from_name']=retweeted_from_name

tweets_df['reply_to_id']=reply_to_id
tweets_df['reply_to_name']=reply_to_name


tweets_df['language']=tweet_language
tweets_df['hashtags']=hashtags



tweets_df.to_csv('demo_data/csv/tweets.csv',index=False)



In [9]:
hashtags=list(filter(lambda word: word != '', hashtags))

hashtags_list_flat = list(itertools.chain(*hashtags))

hashtags_df=pd.DataFrame()
hashtags_df['hashtag']=hashtags_list_flat
hashtags_df['account']=Person
hashtags_df.to_csv('demo_data/csv/hashtags.csv',index=False)


